# Joke Recommender System

## Surprise is a python scikit used to make recommender systems
http://surpriselib.com/

https://github.com/NicolasHug/Surprise

In [1]:
import os
os.chdir("/home/sagar/insofe/acads/CF")

In [18]:
from surprise import Dataset, Reader, KNNWithMeans, KNNWithZScore
from surprise.model_selection import cross_validate
import pandas as pd

## Setting seed for reproducible results

In [3]:
import random
import numpy as np

my_seed = 9873923
random.seed(my_seed)
np.random.seed(my_seed)

## Joke Dataset
http://eigentaste.berkeley.edu/dataset/

150 Jokes

63978 Users

### Reading jokes files
Note - Data is tab seperated

In [4]:
jokes = pd.read_csv("jester_items.tsv",sep="\t",names=["itemID","Joke"],usecols=["Joke"])
jokes.head()

,Joke
0,"A man visits the doctor. The doctor says, ""I h..."
1,This couple had an excellent relationship goin...
2,Q. What's 200 feet long and has 4 teeth? A. Th...
3,Q. What's the difference between a man and a t...
4,Q. What's O. J. Simpson's web address? A. Slas...


### Reading the ratings file

In [5]:
ratings = pd.read_csv("jester_ratings.csv")
ratings.head()

,userID,itemID,rating
0,1,5,0.219
1,1,7,-9.281
2,1,8,-9.281
3,1,13,-6.781
4,1,15,0.875


In [6]:
ratings.describe()

,userID,itemID,rating
count,1.761439e+06,1.761439e+06,1.761439e+06
mean,3.272322e+04,7.071133e+01,1.618602e+00
std,1.828011e+04,4.600790e+01,5.302608e+00
min,1.000000e+00,5.000000e+00,-1.000000e+01
25%,1.720200e+04,2.100000e+01,-2.031000e+00
50%,3.480800e+04,6.900000e+01,2.219000e+00
75%,4.730600e+04,1.120000e+02,5.719000e+00
max,6.397800e+04,1.500000e+02,1.000000e+01


### Defining the parser to read data into surprise dateframe
The parser requires the scale of ratings and the columns in a specific order ['userID', 'itemID', 'rating']

In [7]:
no_of_users = 1000
reader = Reader(rating_scale=(-10, 10))
data = Dataset.load_from_df(ratings[ratings.userID < no_of_users], reader)

## Simulation Parameters
- Algorithm Type
- User-Based vs Item-Based
- Similarity Metric

In [45]:
sim_parameters = {'name': 'cosine', 'user_based': False, 'min_support': 5}
algo = KNNWithMeans(sim_options=sim_parameters, k=9)

## Cross Validation Accuracies

In [46]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=2, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 2 split(s).

                  Fold 1  Fold 2  Mean    Std     
RMSE (testset)    4.8380  4.8327  4.8354  0.0026  
MAE (testset)     3.7009  3.6827  3.6918  0.0091  
Fit time          0.12    0.07    0.09    0.02    
Test time         1.25    1.18    1.21    0.04    


{'fit_time': (0.11516475677490234, 0.07085490226745605),
 'test_mae': array([3.70091223, 3.68272206]),
 'test_rmse': array([4.83802637, 4.83274944]),
 'test_time': (1.2522344589233398, 1.176239252090454)}

## Training the model on complete data

In [10]:
trainset = data.build_full_trainset()
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


## Filtering instances which can be used for predictions

In [11]:
testset = trainset.build_anti_testset()

## Making Predictions

In [12]:
predictions = algo.test(testset)

### Function to calculate top 10 predictions for each user

In [13]:
# Fetching top 10 predictions for each user
from collections import defaultdict
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.
    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.
    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

top_n = get_top_n(predictions, n=10)

### Function to get first n users and corresponding predictions

In [14]:
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

## Top Predictions Matrix

In [15]:
for uid, user_ratings in take(10,top_n.items()):
    print(uid, [iid for (iid, _) in user_ratings])

1 [114, 126, 117, 129, 143, 138, 56, 47, 111, 110]
2 [127, 114, 129, 138, 119, 105, 106, 76, 117, 93]
3 [127, 114, 53, 35, 89, 129, 119, 138, 32, 72]
4 [127, 114, 53, 35, 89, 129, 119, 138, 32, 105]
5 [127, 114, 119, 138, 129, 105, 106, 126, 134, 135]
6 [127, 114, 129, 119, 138, 32, 105, 106, 76, 69]
7 [127, 114, 129, 138, 119, 105, 106, 117, 126, 145]
8 [127, 114, 138, 129, 119, 143, 145, 117, 134, 135]
9 [127, 53, 35, 129, 89, 119, 138, 32, 105, 50]
10 [114, 127, 129, 138, 148, 137, 126, 139, 132, 150]


## Top Jokes for each User

In [16]:
# Printing top predictions
for uid, user_ratings in take(10,top_n.items()):
    print("For User",uid)
    for  (iid, _) in user_ratings:
        print(iid)
        print(jokes.loc[int(iid),"Joke"])
    

For User 1
114
A lady bought a new Lexus. It cost a bundle. Two days later, she brought it back, complaining that the radio was not working. "Madam," said the sales manager, "the audio system in this car is completely automatic. All you need to do is tell it what you want to listen to, and you will hear exactly that!" She drove out, somewhat amazed and a little confused. She looked at the radio and said, "Nelson." The radio responded, "Ricky or Willie?" She was astounded. If she wanted Beethoven, that's what she got. If she wanted Nat King Cole, she got it. She was stopped at a traffic light enjoying "On the Road Again" when the light turned green and she pulled out. Suddenly an enormous sports utility vehicle coming from the street she was crossing sped toward her, obviously not paying attention to the light. She swerved and narrowly missed a collision. "Idiot!" she yelled and, from the radio, "Ladies and gentlemen, the President of the United States."
126
A little boy goes to his dad

KeyError: 'the label [150] is not in the [index]'